<a href="https://colab.research.google.com/github/carlyryan/CIS450/blob/main/Cozy_with_the_Good_Eats_Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Our project uses two datasets:
1. A [Yelp](https://www.yelp.com/dataset) dataset: The Yelp dataset covers 160,585 businesses across 8 metropolitan areas. There are a couple of different tables: business, review, user, checkin, tip, and photo. We’ll probably focus a lot of our attention on the business and review tables. (--> nothing in NYC https://github.com/Yelp/dataset-examples/issues/47)

2. An [Airbnb](http://insideairbnb.com/get-the-data.html) dataset: The AIrBnB dataset, provided by ‘Inside Airbnb’, covers 36,923 locations across the entire NYC metropolitan area. Important columns that we have are latitude and longitude, number of bedrooms, as well as review statistics for a location etc. 


To prepare the datasets for ingestion into the database, we want to:
1. Clean missing / misentered values
2. Detect and solve entity resolution problems
3. Replace categorical variables with numeric indicators for efficiency
4. Export the data

## Cleaning of Data for Stay cozy with good eats

we will look to : 
*   Removal of rows with missing values
*   Replacement of missing values with default values
*   Dectection of entity resolution problems
*   Simple entity resolution
*   Removal of unpaired entities
*   Replacement of categorical variables with indicators
*   Identification of candidate indices
*   Data exportation

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
import math
import requests 
import json

In [ ]:
prefix = '/content/drive'
from google.colab import drive
drive.mount(prefix, force_remount=True)
#success code: 4/1AX4XfWg77OvWTHG6cqauPcN_8-a3FWdeeWTGs1yBFShQYlfue6DcdgYd-70

Mounted at /content/drive


Now, copy the paths to the life expectancy dataset and census dataset to the `le_path` and `census_path` variables respectively in the cell below. Then, run the cell to save your variables.  

In [ ]:
yelp_path = '/content/drive/Shareddrives/CIS550/raw_datasets/yelp.csv' # we dont use this 
airbnb_path = '/content/drive/Shareddrives/CIS550/raw_datasets/airbnb.csv'

In [ ]:
yelp_raw = pd.read_csv(yelp_path) # we dont use this 
airbnb_raw = pd.read_csv(airbnb_path)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,4,7,17,29,49,60,62,79,86,94) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Yelp Data Cleanup
We want to keep all of the columns so we will just look for missing or misentered values, create categorical values, and check for entity resolution. We also only want New York Data.

In [ ]:
yelp_raw.head()

,attributes.Ambience.divey,attributes.Dietary Restrictions.vegan,attributes.Happy Hour,hours.Thursday.open,attributes.Order at Counter,attributes.Hair Types Specialized In.africanamerican,attributes.Hair Types Specialized In.kids,attributes.BYOB,hours.Friday.open,attributes.Good For.latenight,attributes.Outdoor Seating,attributes.Alcohol,attributes.Ambience.classy,attributes.By Appointment Only,attributes.Parking.lot,business_id,attributes.Ambience.touristy,attributes.Corkage,hours.Tuesday.open,attributes.Good For.brunch,categories,attributes.Waiter Service,hours.Monday.open,name,attributes.Parking.street,attributes.Ambience.hipster,attributes.BYOB/Corkage,attributes.Hair Types Specialized In.straightperms,attributes.Music.live,attributes.Dietary Restrictions.dairy-free,attributes.Music.background_music,attributes.Price Range,attributes.Good For.breakfast,attributes.Parking.garage,attributes.Music.karaoke,attributes.Good For Dancing,review_count,attributes.Hair Types Specialized In.asian,state,attributes.Accepts Credit Cards,...,city,attributes.Wheelchair Accessible,attributes.Dietary Restrictions.gluten-free,stars,attributes.Dietary Restrictions.kosher,type,attributes.Caters,attributes.Ambience.intimate,latitude,attributes.Good For.dinner,attributes.Coat Check,longitude,hours.Monday.close,attributes.Hair Types Specialized In.extensions,hours.Tuesday.close,hours.Saturday.close,attributes.Good for Kids,attributes.Parking.validated,hours.Sunday.open,attributes.Accepts Insurance,attributes.Music.dj,attributes.Dietary Restrictions.soy-free,attributes.Has TV,hours.Sunday.close,attributes.Ambience.casual,attributes.Hair Types Specialized In.perms,attributes.Dogs Allowed,attributes.Drive-Thru,attributes.Dietary Restrictions.vegetarian,hours.Wednesday.open,attributes.Noise Level,attributes.Smoking,attributes.Attire,attributes.Hair Types Specialized In.curly,attributes.Good For Groups,neighborhoods,attributes.Open 24 Hours,attributes.Ambience.romantic,attributes.Music.jukebox,attributes.Ambience.upscale
0,False,NaN,NaN,11:00,NaN,NaN,NaN,NaN,11:00,False,False,none,False,NaN,False,5UmKMjUEUNdYWqANhGckJw,False,NaN,11:00,False,"['Fast Food', 'Restaurants']",False,11:00,Mr Hoagie,False,False,NaN,NaN,NaN,NaN,NaN,1.0,False,False,NaN,NaN,4,NaN,PA,True,...,Dravosburg,NaN,NaN,4.5,NaN,business,False,False,40.354327,False,NaN,-79.900706,21:00,NaN,21:00,NaN,True,False,NaN,NaN,NaN,NaN,False,NaN,False,NaN,NaN,False,NaN,11:00,average,NaN,casual,NaN,True,[],NaN,False,NaN,False
1,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,UsFtqoBl7naz8AVUBZMjQQ,NaN,NaN,NaN,NaN,['Nightlife'],NaN,NaN,Clancy's Pub,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,4,NaN,PA,True,...,Dravosburg,NaN,NaN,3.5,NaN,business,NaN,NaN,40.350553,NaN,NaN,-79.886814,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,[],NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3eu6MEFlq2Dg7bQh8QbdOg,NaN,NaN,NaN,NaN,"['Auto Repair', 'Automotive']",NaN,NaN,Joe Cislo's Auto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN,PA,NaN,...,Dravosburg,NaN,NaN,5.0,NaN,business,NaN,NaN,40.350956,NaN,NaN,-79.889059,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,cE27W9VPgO88Qxe4ol6y_g,NaN,NaN,NaN,NaN,"['Active Life', 'Mini Golf', 'Golf']",NaN,NaN,Cool Springs Golf Center,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,NaN,PA,NaN,...,Bethel Park,NaN,NaN,2.5,NaN,business,NaN,NaN,40.354116,NaN,NaN,-80.014660,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,10:00,NaN,NaN,NaN,NaN,10:00,NaN,NaN,NaN,NaN,NaN,False,HZdLhv6COCleJMo7nPl-RA,NaN,NaN,10:00,NaN,"['Shopping', 'Home Services', 'Internet Servic...",NaN,10:00,Verizon,False,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,False,NaN,NaN,5,NaN,PA,False,...,Pittsburgh,NaN,NaN,2.5,NaN,busine

In [ ]:
yelp = yelp_raw[yelp_raw['state']=='NY']
set(yelp_raw['state'])

{'AL',
 'AR',
 'AZ',
 'BW',
 'CA',
 'EDH',
 'ELN',
 'FIF',
 'HAM',
 'IL',
 'KHL',
 'MA',
 'MLN',
 'MN',
 'NC',
 'NM',
 'NTH',
 'NV',
 'NW',
 'ON',
 'OR',
 'PA',
 'QC',
 'RP',
 'SC',
 'SCB',
 'TX',
 'WI',
 nan}

In [ ]:
yelp.columns

Index(['attributes.Ambience.divey', 'attributes.Dietary Restrictions.vegan',
       'attributes.Happy Hour', 'hours.Thursday.open',
       'attributes.Order at Counter',
       'attributes.Hair Types Specialized In.africanamerican',
       'attributes.Hair Types Specialized In.kids', 'attributes.BYOB',
       'hours.Friday.open', 'attributes.Good For.latenight',
       'attributes.Outdoor Seating', 'attributes.Alcohol',
       'attributes.Ambience.classy', 'attributes.By Appointment Only',
       'attributes.Parking.lot', 'business_id', 'attributes.Ambience.touristy',
       'attributes.Corkage', 'hours.Tuesday.open',
       'attributes.Good For.brunch', 'categories', 'attributes.Waiter Service',
       'hours.Monday.open', 'name', 'attributes.Parking.street',
       'attributes.Ambience.hipster', 'attributes.BYOB/Corkage',
       'attributes.Hair Types Specialized In.straightperms',
       'attributes.Music.live', 'attributes.Dietary Restrictions.dairy-free',
       'attributes.Music.

## Important
you can see there is unfortunately no NYC info in the yelp dataset. The following is thus my attempt to grab it directly from the yelp API. It is inspired slightly by the code at the [following link](https://rspiro9.github.io/nyc_restaurant_yelp_and_inspection_analysis)

In [ ]:
pip install requests

In [ ]:
#set(yelp_raw['categories'])

connect to my API key:

In [ ]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)
				
# Pull in keys and specifically draw out the api key. I have removed the specific path to the keys 
# for security purposes:
#keys = get_keys("/Users/3a0K2dom2OuZm69n7rZMR78to5RL7OQgJWqgqIVnKXbFFcPQ4KgMoNWNPrDZ-DVlVUnGB4YuYOob8xU-Fm1WPaK4vLmbaLQ_ljszj7qUPyXNavWHRp-5OgfnPb6OYXYx/yelp_api.json")
api_key = '3a0K2dom2OuZm69n7rZMR78to5RL7OQgJWqgqIVnKXbFFcPQ4KgMoNWNPrDZ-DVlVUnGB4YuYOob8xU-Fm1WPaK4vLmbaLQ_ljszj7qUPyXNavWHRp-5OgfnPb6OYXYx'

# URL to pull data from:
url = 'https://api.yelp.com/v3/businesses/search'

# Identify headers:
headers = {'Authorization': 'Bearer {}'.format(api_key)}

In [ ]:
url = 'https://api.yelp.com/v3/businesses/search'
term = "Restaurants"
location =  'Manhattan, NY'
categories = "(restaurants, All)"
url_params = {
                        'location': location,
                        'term' : term,
                        'categories': categories,
                        'limit': 50,
                    }
response = requests.get(url, headers=headers, params=url_params)
response_json = response.json()

In [ ]:
#create empty dataframe for dataset
column_names = ['business_id', 'categories', 'name',
'postal_code', 'latitude', 'longitude', 'stars',
'review_count', 'price_range', 'hours', 'address', 'neighborhood', 'url']
restaurant =  pd.DataFrame(columns = column_names)

location =  'Manhattan, NY'
categories = "(restaurants, All)"

#looping through different offsets to get restaurants, then I will add to the dataframe
for offset in range(0, 1000, 50):
  search_limit = 50
  term = "Restaurants"
  url_params = {
                  'location': location,
                  'term' : term,
                  'limit': search_limit,
                  'categories': categories,
                  'offset': offset,
              }
  response = requests.get(url, headers=headers, params=url_params)
  if response.status_code == 400:
            print(f'400 Bad Request on offset {offset}')
            break
  if response.status_code == 500:
            print(f'500 Bad Request on offset {offset}')
            break
  response_json = response.json()
  print(response.status_code)
  for i in range(50):
    this_business = response_json['businesses'][i]
    try:
      price = this_business['price'] 
    except:
      price = 'UNKNOWN'
    #price = this_business['price'] if this_business['price'] else 'UNKNOWN'
    new_row = {'business_id': this_business['id'], 
               'categories':this_business['categories'][0]['alias'],
               'name':this_business['name'],
               'postal_code':this_business['location']['zip_code'],
               'latitude': this_business['coordinates']['latitude'],
               'longitude':this_business['coordinates']['longitude'],
               'stars':this_business['rating'],
               'review_count': this_business['review_count'],
               'price_range': price, #'hours': ,
               'address': this_business['location']['address1'], #'neighborhood': , 
               'url': this_business['url']}
    restaurant = restaurant.append(new_row, ignore_index=True)
  #print(response_json['businesses'][0]['name'], response_json, response)
  #print(response)

200
200
200
200
500 Bad Request on offset 200


In [ ]:
restaurant.head()
restaurant.

,business_id,categories,name,postal_code,latitude,longitude,stars,review_count,price_range,hours,address,neighborhood,url
0,DGhWO1sUWydVeR5j5ZZaMw,french,La Grande Boucherie,10019,40.762627,-73.980841,4.5,602,$$$,NaN,145 W 53rd St,NaN,https://www.yelp.com/biz/la-grande-boucherie-n...
1,VvsZAnEwU4c8Xkyrzx05Nw,korean,Anytime,10001,40.747763,-73.986779,4.5,1707,$$,NaN,23 W 32nd St,NaN,https://www.yelp.com/biz/anytime-new-york?adju...
2,B3_K2kUVbYOU0VaLcj_LTw,thai,Thai Villa,10003,40.739020,-73.990650,4.5,3879,$$,NaN,5 E 19th St,NaN,https://www.yelp.com/biz/thai-villa-new-york-2...
3,J3NT61-AH5d5Gu5tFJhYSw,newamerican,The Cabin NYC,10009,40.723930,-73.983830,4.0,351,$$,NaN,205 E 4th St,NaN,https://www.yelp.com/biz/the-cabin-nyc-new-yor...
4,NN3mOWF5e_pnR1ArqM2bHQ,vietnamese,OBAO,10036,40.760654,-73.991332,4.0,2853,$$,NaN,647 9th Ave,NaN,https://www.yelp.com/biz/obao-new-york-3?adjus...


In [ ]:
#create empty dataframe for dataset
column_names = ['review_id',
'stars',
'date',
'useful',
'funny',
'cool',
'review_of']
review =  pd.DataFrame(columns = column_names)

url = 'https://api.yelp.com/v3/businesses/{id}/reviews'
#loop through restaurants in df
for restaurant_id in restaurant['business_id']: 
  search_limit = 50
  url = 'https://api.yelp.com/v3/businesses/' + restaurant_id + '/reviews'
  url_params = {
                'limit': search_limit,
              }
  response = requests.get(url, headers=headers, params=url_params)
  if response.status_code == 400:
            print(f'400 Bad Request')
            break
  if response.status_code == 500:
            print(f'500 Bad Request ')
            break
  response_json = response.json()
  for i in range(len(response_json['reviews'])):
    this_review = response_json['reviews'][i]
    #price = this_business['price'] if this_business['price'] else 'UNKNOWN'
    new_row = { ADD HERE}
    review = review.append(new_row, ignore_index=True)
  #print(response_json['businesses'][0]['name'], response_json, response)
  #print(response)

In [ ]:
url = 'https://api.yelp.com/v3/businesses/' + 'DGhWO1sUWydVeR5j5ZZaMw' + '/reviews'

search_limit = 50
url_params = {
                'limit': search_limit,
}
response = requests.get(url, headers=headers, params=url_params)
response_json = response.json()
response_json

{'possible_languages': ['en', 'es', 'fr'],
 'reviews': [{'id': 'N3weP146Gwqe4k3Jxj-Viw',
   'rating': 5,
   'text': 'Beautiful place that is just amazing and  gigantic.  Good ambiance with a  tremendous bar scene.  Great for people watching and you can meet really super...',
   'time_created': '2021-10-19 20:15:46',
   'url': 'https://www.yelp.com/biz/la-grande-boucherie-new-york-2?adjust_creative=LfJ5XuLXZYLlnozpquhGdA&hrid=N3weP146Gwqe4k3Jxj-Viw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=LfJ5XuLXZYLlnozpquhGdA',
   'user': {'id': 'X87YR8YLa2jiuGmr8oCNjA',
    'image_url': 'https://s3-media1.fl.yelpcdn.com/photo/l_mxRdr7by15xqzJKJN-gA/o.jpg',
    'name': 'Antonio P.',
    'profile_url': 'https://www.yelp.com/user_details?userid=X87YR8YLa2jiuGmr8oCNjA'}},
  {'id': '4fsnqFUDN2Xql0xGxIPTbw',
   'rating': 5,
   'text': 'Such amazing combination of a beautiful ambiance and menu. Highly recommended. The drinks were fantastic, with exquisite food. Our server was v

## What we need now:
1. decide if we want reviews table and import that
2. figure out hours and neighborhood

# Airbnb Data Cleanup
For the Airbnb Data, we want to remove a lot of columns and only keep a few. We also want to have a host table. 

We will also look for missing or misentered values, create categorical values, and check for entity resolution. We also only want New York Data.

In [ ]:
airbnb_raw.head()

### Colums that the Airbnb csv started with : 

```
Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month'],
      dtype='object')
```

### Our DDL: 
```
CREATE TABLE Airbnb(
Id int PRIMARY KEY,
Listing_url varchar(20),
Latitude float,
Longitude float,
bathrooms float, 
bedrooms int,
price float, 
review_scores_value float, 
review_scores_count int, 
neighborhood varchar(40))
foreign key host_id references Host.host_id

CREATE TABLE Host(
host_id int PRIMARY KEY,
...

```


### Columns we would like to thus keep: 
```
Airbnb: [
'id', 'listing_url', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id'
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'number_of_reviews', 'review_scores_rating',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location', 'instant_bookable']
# id is the primary key
# host_id is a foreign key to Host

Host: [
  'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost',
       'host_neighbourhood', 
       'host_total_listings_count'
]
# host_id is the primary key
```

In [ ]:
airbnb = airbnb_raw[[
'id', 'listing_url', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'number_of_reviews', 'review_scores_rating',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location', 'instant_bookable']]
host = airbnb_raw[[
  'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost',
       'host_neighbourhood', 
       'host_total_listings_count'
]]
host.head()

#Cleaning Host

In [ ]:
set(host.host_response_time)
host['host_response_rate'].isnull

In [ ]:
host.iloc[500]

In [ ]:
list_non_null_host_fields = ['host_is_superhost','host_id', 'host_url',
       'host_name', 'host_since', 'host_location',
       'host_is_superhost',
       'host_neighbourhood', 
       'host_total_listings_count', 'host_response_rate', 
       'host_acceptance_rate']
missing_hosts = host.isnull()
missing_hosts = missing_hosts[list_non_null_host_fields].any(axis=1)
host = host.loc[(~missing_hosts).values, :]

In [ ]:
host['host_id'].isnull().any().any()

We now want to determine what types we want for each column. Below we convert to ints and strings. We also convert the percent entries into ints. We also convert 'host_is_superhost' into a boolean. Additionally we verify that 'host_response_time' is prepped to become an enum of type {'a few days or more', 'within a day', 'within a few hours', 'within an hour'}


In [ ]:
def perc_to_int(x): 
  return int(x[:-1])
#int(host.loc[500]['host_response_rate'][:-1])

# here we are converting the percentages to ints without the '%'
host['host_response_rate'] = host['host_response_rate'].apply(lambda x : int(x[:-1]))
host['host_acceptance_rate'] = host['host_acceptance_rate'].apply(lambda x : int(x[:-1]))

# here we are converting the number of listings to ints
host['host_total_listings_count'] = host['host_total_listings_count'].apply(lambda x : int(x))

# here we will convert to strings 
convert_to_str = ['host_url','host_name','host_location','host_about','host_neighbourhood']
for col in convert_to_str:  
  host[col] = host[col].apply(lambda x : str(x))

#host['host_url'] = host['host_url'].apply(lambda x : str(x))
#host['host_name'] = host['host_name'].apply(lambda x : str(x))
#host['host_location'] = host['host_location'].apply(lambda x : str(x))
#host['host_about'] = host['host_about'].apply(lambda x : str(x))
#host['host_neighbourhood'] = host['host_neighbourhood'].apply(lambda x : str(x))

# 'Host_response_time' is already an enum :
set(host['host_response_time'])

In [ ]:
# make 'host_is_superhost' a boolean
host['host_is_superhost'] = host['host_is_superhost'].apply(lambda x : False if x == 'f' else True)

In [ ]:
host.head()

#Cleaning Airbnb

here we will remove nulls from the columns where we deem them not appropriate. 

We will also rename 2 columns to 'neighbourhood' and 'borough'

We will then change data types

In [ ]:
airbnb.head()

In [ ]:
#rename columns 'neighbourhood_cleansed', 'neighbourhood_group_cleansed'
airbnb = airbnb.rename(columns={'neighbourhood_cleansed': 'neighborhood', 'neighbourhood_group_cleansed': 'borough', 'bathrooms_text': 'bathrooms_details'})

In [ ]:
list_non_null_airbnb_fields = ['id', 'listing_url', 'name', 'picture_url', 'host_id',
'neighborhood', 'borough', 'latitude',
'longitude', 'property_type', 'room_type', 'accommodates',
'bathrooms_details', 'bedrooms', 'beds', 'amenities', 'price',
'minimum_nights', 'maximum_nights', 'number_of_reviews', 'review_scores_rating',
'review_scores_cleanliness', 'review_scores_checkin',
'review_scores_communication', 'review_scores_location', 'instant_bookable']
missing_airbnb = airbnb.isnull()
missing_airbnb = missing_airbnb[list_non_null_airbnb_fields].any(axis=1)
airbnb.size #1033844
airbnb = airbnb.loc[(~missing_airbnb).values, :]
airbnb.size #669284

In [ ]:
# making a dict to convert bathroom input to ints 
bathroom_dict = {'0 baths': 0,
 '0 shared baths': 0,
 '1 bath' : 1,
 '1 private bath': 1,
 '1 shared bath': 1,
 '1.5 baths': 1.5,
 '1.5 shared baths': 1.5,
 '15.5 baths': 15.5,
 '2 baths': 2,
 '2 shared baths': 2,
 '2.5 baths': 2.5,
 '2.5 shared baths': 2.5,
 '3 baths': 3,
 '3 shared baths': 3,
 '3.5 baths': 3.5,
 '3.5 shared baths': 3.5,
 '4 baths': 4,
 '4 shared baths': 4,
 '4.5 baths': 4.5,
 '5 baths': 5,
 '5.5 baths': 5.5,
 '6 baths': 6,
 '6 shared baths': 6,
 '6.5 baths': 6.5,
 '7 baths': 7,
 '8 baths': 8,
 'Half-bath': 0.5,
 'Private half-bath': 0.5,
 'Shared half-bath': 0.5}

In [ ]:
# convert to floats
#'id', 'host_id', 'amenities', 'price', 'instant_bookable'
convert_to_float = ['review_scores_rating',
'review_scores_cleanliness', 'review_scores_checkin',
'review_scores_communication', 'latitude',
'longitude', 'review_scores_location']
convert_to_int = ['minimum_nights', 'maximum_nights', 'number_of_reviews', 'bedrooms', 'beds', 'accommodates', 'id', 'host_id']
convert_to_str = ['listing_url', 'name', 'description',
'neighborhood_overview', 'picture_url', 'neighborhood', 'property_type', 'bathrooms_details']

for col in convert_to_float:  
  airbnb[col] = airbnb[col].apply(lambda x : float(x))
for col in convert_to_int:  
  airbnb[col] = airbnb[col].apply(lambda x : int(x))
for col in convert_to_str:  
  airbnb[col] = airbnb[col].apply(lambda x : str(x))

# here we are converting the prices to floats without the '$'
airbnb['price'] = airbnb['price'].apply(lambda x : float(x[1:].replace(',','')))

# here we are converting the bathrooms_text to just an int
airbnb['bathrooms'] = airbnb['bathrooms_details'].apply(lambda x : bathroom_dict[x])

# 'room_type' is already an enum :
# {'Entire home/apt', 'Hotel room', 'Private room', 'Shared room'}
set(airbnb['room_type'])
# 'borough' is already an enum :
# {'Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island'}
set(airbnb['borough'])

# make 'instant_bookable' a boolean
airbnb['instant_bookable'] = airbnb['instant_bookable'].apply(lambda x : False if x == 'f' else True)

In [ ]:
set(airbnb['instant_bookable'])

In [ ]:
host.dtypes

We finish with the following for **airbnb**
```
id                               int64 PRIMARY KEY
listing_url                     object
name                            object
description                     object CAN BE NULL
neighborhood_overview           object CAN BE NULL
picture_url                     object
host_id                          int64 FOREIGN KEY to Host
neighborhood                    object
borough                         object ENUM
latitude                       float64
longitude                      float64
property_type                   object
room_type                       object ENUM
accommodates                     int64
bathrooms_details               object
bedrooms                         int64
beds                             int64
amenities                       object
price                          float64
minimum_nights                   int64
maximum_nights                   int64
number_of_reviews                int64
review_scores_rating           float64
review_scores_cleanliness      float64
review_scores_checkin          float64
review_scores_communication    float64
review_scores_location         float64
instant_bookable                  bool
bathrooms                      float64
```
We finish with the following for **host**


```
host_id                       int64 PRIMARY KEY
host_url                     object
host_name                    object
host_since                   object
host_location                object
host_about                   object CAN BE NULL
host_response_time           object ENUM
host_response_rate            int64
host_acceptance_rate          int64
host_is_superhost              bool
host_neighbourhood           object
host_total_listings_count     int64

```




## Entity Resolution

Now that we've handled missing values in both datasets, we turn our attention to performing entity resolution on the entities common to both. In this case, those common entities are countries.

To perform entity resolution, we will:
1. Determine whether both datasets use the same names to refer to all countries
2. Edit the names in one dataset to match the other, if necessary

In the general case, you may also need to detect when datasets refer to different entities using the same name/ID and disambiguate these references. This may happen when handling datasets that contain multiple people with the same name, for example. But we don't need to worry about it here because the names of countries are well-known and distict. 


### Detect Inconsistent Names
Let's compile a list of all country names in both datasets, then inspect it for repetitions. 

First, we extract the unique names of countries from both datasets.

In [ ]:
census_countries = census["native-country"].unique()
le_countries = le["country"].unique()


Now, we combine these into one set.

In [ ]:
countries = set(census_countries.tolist() + le_countries.tolist())

Let's inspect the contents of the set. The output is automatically sorted.

In [ ]:
countries

Immediately, we see that many of the names have spaces prepended to them that appear to be preventing matches. Let's print the lists of names from each dataset separately to see where these are coming from. 

In [ ]:
print("Census Countries")
print(census_countries)
print("")
print("LE Countries")
print(le_countries)

The output above indicates the names with spaces are coming from the census dataset. So let's remove those spaces, recompile the list, and take look for countries that appear twice in the output.

In [ ]:
fixed_census_countries = list()
for country in census_countries.tolist():
  if country[0] == " ":
    fixed_census_countries.append(country[1:])
countries = set(fixed_census_countries + le_countries.tolist())
countries

The output contains 9 countries with multiple names:

*   "United States of America" and "United-States"
*   "Trinidad and Tobago" and "Trinidad&Tobago"
*   "Dominican Republic" and "Dominican-Republic"
*   "El-Salvador" and "El Salvador"
*   "Columbia" and "Colombia"
*   "Netherlands" and "Holand-Netherlands"
*   "Laos" and "Lao People's Democratic Republic"
*   "Iran" and "Iran (Islamic Republic of)"
*   "Viet Nam" and "Vietnam"

Relatedly, the output contains an entry for the United Kingdom, as well as entries for Scotland and England, which are part of the UK. 

*Republic of Korea and the Democratic People's Republic of Korea are different countries. Congo and Democratic Republic of the Congo are also different.* 

### Resolve Inconsistent Names
Now, let's fix the problems the we uncovered in the previous step.

First, we trim the whitespace off the start and end of the country names in both datasets to handle that space issue.

In [ ]:
census["native-country"] = census["native-country"].str.strip()
le["country"] = le["country"].str.strip()

For each country with multiple names, we need to purge one of the names from the dataset. We'll do this by replacing all usages of one name with the other. 

We'll also replace all substitute all usages Scotland and England for the U.K. This solution sacrifices some precision, but at least we avoid the incorrect conclusion that there's no valid counterpart for rows containing Scotland and England.  

To accomplish this, we first create a `Series` of the countries with multiple names, where of each element is the name we'll purge, and the value is name we'll keep.

In [ ]:
name_map = {
     "United-States": "United States of America",
     "Trinadad&Tobago": "Trinidad and Tobago",
     "Dominican-Republic": "Dominican Republic",
     "Columbia" : "Colombia",
     "Lao People's Democratic Republic": "Laos",
     "Scotland": "United Kingdom of Great Britain and Northern Ireland",
     "England": "United Kingdom of Great Britain and Northern Ireland",
     "Iran (Islamic Republic of)": "Iran",
     "Holand-Netherlands": "Netherlands",
     "Viet Nam" : "Vietnam",
     "El-Salvador": "El Salvador"
}
name_series = pd.Series(data=list(name_map.values()), index=list(name_map.keys()))
name_series

Next, we find the indices of all usages of the names we're purging in both datasets

In [ ]:
purge_list = list(name_map.keys())
census_idx = census.index[census["native-country"].isin(purge_list)]
le_idx = le.index[le["country"].isin(purge_list)]

Using these indices, we extract the names that we need to update as `Series`

In [ ]:
census_problems = census.loc[census_idx, "native-country"]
le_problems = le.loc[le_idx, "country"]

Next, we replace the problematic names in these `Series` with the appropriate counterparts by indexing `name_series` with them. 

*The operations in the cell below replace the names correctly because `name_series` maps each name we wanted to purge to the we wanted to replace it with.*

In [ ]:
census_fixed = name_series.loc[census_problems.values].values
le_fixed = name_series.loc[le_problems.values].values 

Finally, we update the dataframes with the fixed names.

In [ ]:
census.loc[census_idx, "native-country"] = census_fixed
le.loc[le_idx, "country"] = le_fixed

### Remove Unpaired Entities
In some cases, you may want to exclude entities that only appear in one dataset or the other from your database. Let's suppose that's the case here and remove all countries that only appear in one dataset or the other. 

As before, we first extract the full list of countries found in both datasets.

In [ ]:
census_countries = census["native-country"].unique().tolist()
le_countries = le["country"].unique().tolist()

Then, we convert these lists to sets and use set-difference operations to find the countries that only appear in one set. 

In [ ]:
census_countries = set(census_countries)
le_countries = set(le_countries)
cen_diff = census_countries.difference(le_countries)
le_diff = le_countries.difference(census_countries)
total_diff = list(cen_diff) + list(le_diff)

Finally, we drop all rows from both datasets that contain countries that match any of the countries in this list. 

In [ ]:
le = le.loc[~le["country"].isin(total_diff), :]
census = census.loc[~census["native-country"].isin(total_diff), :]

## Replace Categorical Variables with Indicators

Depending on your application, you may want to replace some of your text-based categorical variables with numeric equivalents. This substitution can reduce the runtime of some queries and decrease the your tables require. It can be especially fruitful for columns you plan to use as indices.  

Let's see an example. Below, we'll convert the columns containing countries in our datasets to numeric equivalents. 

First, we create a `Series` that maps each country name to an integer. 


In [ ]:
all_countries = le["country"].unique()
country_codes = pd.Series(index=all_countries, data=np.arange(len(all_countries)))
country_codes

Next, we use this `Series` to map all countries in both datasets to the corresponding integers by indexing the `Series` with the names of the countries in the datasets. 

In [ ]:
le["country"] = country_codes.loc[le["country"]].values
census["native-country"] = country_codes.loc[census["native-country"]].values

Finally, we convert the columns to integer types. 

In [ ]:
le["country"] = le["country"].astype(np.int64)
census["native-country"] = census["native-country"].astype(np.int64)


## Find an Index

Before ingesting our data into the database, we need to find a unique index for each table. Let's do this for the life expectancy dataset. 

### Single Column Index
The fastest way to determine whether any single column is unique is to check whether the number of unique values in the candidate column equals the number of elements. For example, let's find out whether `country` could be the index.  

In [ ]:
len(le["country"].unique()) == len(le["country"])

### Multi-Column Index

When there's not an individual column that can act as an index, we search for combinations of columns that can make a unique index when combined. 

To check a candidate set of columns:
1. Call `DataFrame.groupby()` on the list of candidate columns. *This creates a group of rows for each unique combination of candidate olumn values that appears in the Dataframe*
2. Call `GroupBy.size()` on the resulting grouped dataframe. *This counts the number of rows in each group*
3. Check whether every group has exactly 1 row. 

We use this procedure below to check whether `country` and `year` can function as a joint index. 

In [ ]:
# Step 1: Group dataframe by candidate columns 
grouped_le = le.groupby(["country", "year"]) 
# Step 2: Count rows in each group
counts = grouped_le.size()
print(counts)
# Step 3: Check whether every value equals 1
(counts == 1).all()

Great! This means no country-year pair appears more than once in the life expectancy table, so we can use `country` and `year` in combination as the index. 

For the census dataset, you'd need every column to create a unique index (Check this for yourself). So we'll just plan to use the arbitrary, unique integers `census.index` as our table index. 

## Export Data
After cleaning our datasets, resolving entity resolution problems, and choosing indices, we're ready to ingest our the data into our database.

In the past, students have found using Python for data ingestion slow and frustrating, so we won't populate the database here. Instead, we'll export both datasets and the country codes to CSVs. Then, we'll show you how to ingest these CSVs into your database using MySQL Workbench in the tutorial on data ingestion. 


First, convert the `Series` of country codes to an equivalent `DataFrame`

In [ ]:
country_codes = pd.DataFrame(data={'country': country_codes.index, 'code': country_codes.values})

Next, we use `DataFrame.to_csv` to write each dataset to a CSV file with a descriptive name. 

*For `le` and `country_codes`, we set `index` to `False` because the indices of the `DataFrames` are meaningless integers that we don't need in our tables. For `census`, we rename the index and include it in the output because we decided  to use it as our table index, even though it's arbitrary.*

In [ ]:
le.to_csv("le.csv", index=False)
country_codes.to_csv("country_codes.csv", index=False)
census.index.name = "id"
census.to_csv("census.csv", index=True)

Finally, we download these files to our local machine, so we can put them into MySQL Workbench later. 

In [ ]:
from google.colab import files
files.download('census.csv') 
files.download("country_codes.csv")
files.download("le.csv")

## Exercises
Check out [these exercises](https://drive.google.com/open?id=1kjLaYC_KJUlltm-iAzgF5VmHgb7Uer0z) to practice the processing techniques you learned above!